In [4]:
# Import the required packages
import sqlite3

In [5]:
# Connect to the database
conn = sqlite3.connect('Q2.db')
cursor = conn.cursor()

# Create the first table for the survey data
cursor.execute('''
    CREATE TABLE IF NOT EXISTS survey_data (
        id INT,
        agerang INT,
        sex INT,
        degree INT
    )
''')

# Create the second table
cursor.execute('''
     CREATE TABLE IF NOT EXISTS keys (
        lookup INT,
        descr TEXT
    )
''')

# Insert data into survey_data table
# Read the data from the text file
with open('data.txt', 'r') as file:
    next(file)
    
    # Loop through the file and insert the data into the database
    for line in file:
        # Split the line into columns
        columns = line.strip().split('\t')
        
        # Insert the data into the table
        cursor.execute('INSERT INTO survey_data VALUES (?, ?, ?, ?)', columns)


# Insert data into keys table
with open('keys.txt', 'r') as file:
    next(file)
    
    # Loop through the file and insert the data into the database
    for line in file:
        # Split the line into columns
        columns = line.strip().split('\t')
        
        # Insert the data into the table
        cursor.execute('INSERT INTO keys VALUES (?, ?)', columns)

# Commit the changes to the database
conn.commit()

# Close the cursor and the database connection
cursor.close()
conn.close()

In this question 2 tables (collections of data) are given in the data.txt and keys.txt files. The goal of this question was to combine the tables based on the ages, sex and degrees of the individuals while also giving a count of the group and show the id of each person in that group. E.g (Agerang: 20, Count: 18, Id:2,11,36,42,44,45,51,52,57,65,67,68,69,79,86,88,90,91).

In this notebook the problem is solved in 2 ways:
- Firstly it is solved through iterative problem solving with SQL to achieve the result (Problem solving)
- Secondly the problem is solved in a simplified way by using a more compact SQL statement (Effecient SQL code)

# Method 1

In [7]:
# Connect to the database
conn = sqlite3.connect('Q2.db')
cursor = conn.cursor()

# View the contents of survey_data table
cursor.execute('SELECT * FROM survey_data')
table1_data = cursor.fetchall()
print("Survey Data Table:")
for row in table1_data:
    print(row)
print('')

# View the contents of keys table
cursor.execute('SELECT * FROM keys')
table2_data = cursor.fetchall()
print("Keys table:")
for row in table2_data:
    print(row)
print('')

# Close the cursor and the database connection
cursor.close()
conn.close()

Survey Data Table:
(1, 3, 6, 7)
(2, 1, 6, 7)
(3, 2, 5, 7)
(4, 3, 6, 10)
(5, 2, 5, 10)
(6, 3, 6, 9)
(7, 4, 5, 9)
(8, 4, 6, 9)
(9, 3, 6, 9)
(10, 2, 6, 10)
(11, 1, 5, 8)
(12, 2, 6, 8)
(13, 3, 6, 7)
(14, 3, 5, 7)
(15, 4, 6, 9)
(16, 4, 5, 8)
(17, 4, 5, 7)
(18, 4, 5, 10)
(19, 2, 6, 10)
(20, 3, 6, 8)
(21, 4, 6, 8)
(22, 4, 5, 8)
(23, 4, 5, 8)
(24, 3, 5, 9)
(25, 2, 6, 8)
(26, 2, 6, 9)
(27, 2, 6, 10)
(28, 4, 6, 9)
(29, 2, 6, 10)
(30, 2, 6, 8)
(31, 4, 6, 9)
(32, 2, 5, 9)
(33, 4, 5, 8)
(34, 4, 5, 9)
(35, 4, 5, 9)
(36, 1, 5, 7)
(37, 3, 6, 9)
(38, 2, 5, 7)
(39, 3, 5, 10)
(40, 2, 6, 7)
(41, 4, 5, 10)
(42, 1, 5, 10)
(43, 3, 6, 7)
(44, 1, 6, 8)
(45, 1, 6, 10)
(46, 3, 5, 7)
(47, 4, 6, 8)
(48, 3, 6, 10)
(49, 4, 5, 7)
(50, 3, 6, 10)
(51, 1, 5, 9)
(52, 1, 6, 8)
(53, 2, 5, 8)
(54, 4, 6, 9)
(55, 4, 6, 7)
(56, 3, 6, 10)
(57, 1, 6, 9)
(58, 2, 5, 8)
(59, 4, 6, 7)
(60, 2, 5, 9)
(61, 3, 5, 10)
(62, 4, 5, 9)
(63, 2, 5, 8)
(64, 2, 5, 8)
(65, 1, 6, 10)
(66, 3, 6, 10)
(67, 1, 6, 9)
(68, 1, 5, 7)
(69, 1, 6, 7)
(70, 4,

In [8]:
# Execute the SQL query to determine unique entries, count occurrences, and group IDs for agerang
conn = sqlite3.connect('Q2.db')
cursor = conn.cursor()
cursor.execute('''
    SELECT agerang, COUNT(*) AS count, GROUP_CONCAT(id) AS id_list
    FROM survey_data
    GROUP BY agerang
''')

# Fetch all rows from the result set
rows = cursor.fetchall()

# Create a new table to store the results
cursor.execute('''
    CREATE TABLE IF NOT EXISTS agerang_count (
        agerang INTEGER,
        count INTEGER,
        id_list TEXT
    )
''')

# Insert the rows into the new table
cursor.executemany('''
    INSERT INTO agerang_count (agerang, count, id_list)
    VALUES (?, ?, ?)
''', rows)

# Execute the SQL statement to update the values for the agerang column
cursor.execute('''
    UPDATE agerang_count
    SET agerang = (
        SELECT descr
        FROM keys
        WHERE agerang_count.agerang = keys.lookup
    );
''')

# View the contents of agerang counts
cursor.execute('SELECT * FROM agerang_count')
table_data = cursor.fetchall()
print("Agerang table:")
print("Agerang, Count, IDs")
for row in table_data:
    print(row)
print('')

# Commit the changes to the database
conn.commit()
    
# Close the cursor and the database connection
cursor.close()
conn.close()

Agerang table:
Agerang, Count, IDs
('20-29', 18, '2,11,36,42,44,45,51,52,57,65,67,68,69,79,86,88,90,91')
('30-39', 24, '3,5,10,12,19,25,26,27,29,30,32,38,40,53,58,60,63,64,75,83,85,87,93,94')
('40-49', 26, '1,4,6,9,13,14,20,24,37,39,43,46,48,50,56,61,66,71,73,74,80,89,92,95,96,98')
('50-59', 32, '7,8,15,16,17,18,21,22,23,28,31,33,34,35,41,47,49,54,55,59,62,70,72,76,77,78,81,82,84,97,99,100')



In [13]:
# Execute the SQL query to determine unique entries, count occurrences, and group IDs for sex
conn = sqlite3.connect('Q2.db')
cursor = conn.cursor()
cursor.execute('''
    SELECT sex, COUNT(*) AS count, GROUP_CONCAT(id) AS id_list
    FROM survey_data
    GROUP BY sex
''')

# Fetch all rows from the result set
rows = cursor.fetchall()

# Create a new table to store the results
cursor.execute('''
    CREATE TABLE IF NOT EXISTS sex_count (
        sex INTEGER,
        count INTEGER,
        id_list TEXT
    )
''')

# Insert the rows into the new table
cursor.executemany('''
    INSERT INTO sex_count (sex, count, id_list)
    VALUES (?, ?, ?)
''', rows)

# Execute the SQL statement to update the values for the sex column
cursor.execute('''
    UPDATE sex_count
    SET sex = (
        SELECT descr
        FROM keys
        WHERE sex_count.sex = keys.lookup
    );
''')

# View the contents of sex counts
cursor.execute('SELECT * FROM sex_count')
table_data = cursor.fetchall()
print("Sex table:")
print("Sex, Count, IDs")
for row in table_data:
    print(row)
print('')

# Commit the changes to the database
conn.commit()

# Close the cursor and the database connection
cursor.close()
conn.close()

Sex table:
Sex, Count, IDs
(None, 46, '3,5,7,11,14,16,17,18,22,23,24,32,33,34,35,36,38,39,41,42,46,49,51,53,58,60,61,62,63,64,68,71,72,75,77,81,83,86,89,90,92,93,95,98,99,100')
(None, 54, '1,2,4,6,8,9,10,12,13,15,19,20,21,25,26,27,28,29,30,31,37,40,43,44,45,47,48,50,52,54,55,56,57,59,65,66,67,69,70,73,74,76,78,79,80,82,84,85,87,88,91,94,96,97')
('male', 46, '3,5,7,11,14,16,17,18,22,23,24,32,33,34,35,36,38,39,41,42,46,49,51,53,58,60,61,62,63,64,68,71,72,75,77,81,83,86,89,90,92,93,95,98,99,100')
('female', 54, '1,2,4,6,8,9,10,12,13,15,19,20,21,25,26,27,28,29,30,31,37,40,43,44,45,47,48,50,52,54,55,56,57,59,65,66,67,69,70,73,74,76,78,79,80,82,84,85,87,88,91,94,96,97')



In [14]:
# Execute the SQL query to determine unique entries, count occurrences, and group IDs for degree
conn = sqlite3.connect('Q2.db')
cursor = conn.cursor()
cursor.execute('''
    SELECT degree, COUNT(*) AS count, GROUP_CONCAT(id) AS id_list
    FROM survey_data
    GROUP BY degree
''')

# Fetch all rows from the result set
rows = cursor.fetchall()
   
# Create a new table to store the results
cursor.execute('''
    CREATE TABLE IF NOT EXISTS degree_count (
        degree INTEGER,
        count INTEGER,
        id_list TEXT
    )
''')

# Insert the rows into the new table
cursor.executemany('''
    INSERT INTO degree_count (degree, count, id_list)
    VALUES (?, ?, ?)
''', rows)

# Execute the SQL statement to update the values for the degree column
cursor.execute('''
    UPDATE degree_count
    SET degree = (
        SELECT descr
        FROM keys
        WHERE degree_count.degree = keys.lookup
    );
''')

# View the contents of degree counts
cursor.execute('SELECT * FROM degree_count')
table_data = cursor.fetchall()
print("Degree table:")
print("Degree, Count, IDs")
for row in table_data:
    print(row)
print('')

# Commit the changes to the database
conn.commit()

# Close the cursor and the database connection
cursor.close()
conn.close()

Degree table:
Degree, Count, IDs
('none', 23, '1,2,3,13,14,17,36,38,40,43,46,49,55,59,68,69,71,79,82,86,88,94,97')
('primary', 24, '11,12,16,20,21,22,23,25,30,33,44,47,52,53,58,63,64,75,78,83,89,90,93,99')
('secondary', 28, '6,7,8,9,15,24,26,28,31,32,34,35,37,51,54,57,60,62,67,73,74,76,77,80,85,96,98,100')
('tertiary', 25, '4,5,10,18,19,27,29,39,41,42,45,48,50,56,61,65,66,70,72,81,84,87,91,92,95')



# Method 2

In [16]:
# Connect to the database
conn = sqlite3.connect('Q2.db')
cursor = conn.cursor()

# Collect all the agrang data, count the entries and group them by using the lookup column in the keys file
cursor.execute('''SELECT keys.descr AS agerang, COUNT(*) AS count, GROUP_CONCAT(survey_data.id) AS id_list
FROM survey_data
JOIN keys ON survey_data.agerang = keys.lookup
GROUP BY survey_data.agerang''')

# Show the grouped table for agerang
for i in cursor.fetchall():
    print(i)

# Collect all the sex data, count the entries and group them by using the lookup column in the keys file
cursor.execute('''SELECT keys.descr AS sex, COUNT(*) AS count, GROUP_CONCAT(survey_data.id) AS id_list
FROM survey_data
JOIN keys ON survey_data.sex = keys.lookup
GROUP BY survey_data.sex''')

# Show the grouped table for sex
print('')
for i in cursor.fetchall():
    print(i)

# Collect all the degree data, count the entries and group them by using the lookup column in the keys file
cursor.execute('''SELECT keys.descr AS degree, COUNT(*) AS count, GROUP_CONCAT(survey_data.id) AS id_list
FROM survey_data
JOIN keys ON survey_data.degree = keys.lookup
GROUP BY survey_data.degree''')

# Show the grouped table for degree
print('')
for i in cursor.fetchall():
    print(i)

# Close the cursor and the database connection
cursor.close()
conn.close()

('20-29', 18, '2,11,36,42,44,45,51,52,57,65,67,68,69,79,86,88,90,91')
('30-39', 24, '3,5,10,12,19,25,26,27,29,30,32,38,40,53,58,60,63,64,75,83,85,87,93,94')
('40-49', 26, '1,4,6,9,13,14,20,24,37,39,43,46,48,50,56,61,66,71,73,74,80,89,92,95,96,98')
('50-59', 32, '7,8,15,16,17,18,21,22,23,28,31,33,34,35,41,47,49,54,55,59,62,70,72,76,77,78,81,82,84,97,99,100')

('male', 46, '3,5,7,11,14,16,17,18,22,23,24,32,33,34,35,36,38,39,41,42,46,49,51,53,58,60,61,62,63,64,68,71,72,75,77,81,83,86,89,90,92,93,95,98,99,100')
('female', 54, '1,2,4,6,8,9,10,12,13,15,19,20,21,25,26,27,28,29,30,31,37,40,43,44,45,47,48,50,52,54,55,56,57,59,65,66,67,69,70,73,74,76,78,79,80,82,84,85,87,88,91,94,96,97')

('none', 23, '1,2,3,13,14,17,36,38,40,43,46,49,55,59,68,69,71,79,82,86,88,94,97')
('primary', 24, '11,12,16,20,21,22,23,25,30,33,44,47,52,53,58,63,64,75,78,83,89,90,93,99')
('secondary', 28, '6,7,8,9,15,24,26,28,31,32,34,35,37,51,54,57,60,62,67,73,74,76,77,80,85,96,98,100')
('tertiary', 25, '4,5,10,18,19,27,29,